In [4]:
!pip install cvc5

In [1]:
from cvc5 import Solver, Kind

# Initialize cvc5 solver
solver = Solver()
solver.setOption("produce-models", "true")
solver.setLogic("QF_BV")  # Quantifier-free bit-vector logic

# Define TCP Header Fields as symbolic bit-vectors


# 16-bit source port and destination port
source_port = solver.mkConst(solver.mkBitVectorSort(16), "source_port")
dest_port = solver.mkConst(solver.mkBitVectorSort(16), "dest_port")

# 32-bit sequence number and acknowledgment number
sequence_number = solver.mkConst(solver.mkBitVectorSort(32), "sequence_number")
ack_number = solver.mkConst(solver.mkBitVectorSort(32), "ack_number")

# 8-bit data offset + reserved + flags, and flags (for simplicity, we will treat these as two separate bytes)
data_offset_reserved_flags = solver.mkConst(solver.mkBitVectorSort(8), "data_offset_reserved_flags")
flags = solver.mkConst(solver.mkBitVectorSort(8), "flags")

# 16-bit window size, checksum, and urgent pointer
window_size = solver.mkConst(solver.mkBitVectorSort(16), "window_size")
checksum = solver.mkConst(solver.mkBitVectorSort(16), "checksum")
urgent_pointer = solver.mkConst(solver.mkBitVectorSort(16), "urgent_pointer")


#test

# Define flag bit values
URG_FLAG = solver.mkBitVector(8, 0x20)  # URG flag (0b00100000)
ACK_FLAG = solver.mkBitVector(8, 0x10)  # ACK flag (0b00010000)
PSH_FLAG = solver.mkBitVector(8, 0x08)  # PSH flag (0b00001000)
RST_FLAG = solver.mkBitVector(8, 0x04)  # RST flag (0b00000100)
SYN_FLAG = solver.mkBitVector(8, 0x02)  # SYN flag (0b00000010)
FIN_FLAG = solver.mkBitVector(8, 0x01)  # FIN flag (0b00000001)

# Example: Constraint for SYN-ACK packet
# Combine SYN and ACK flags using bitwise OR to create SYN-ACK flag
syn_ack_flags = solver.mkTerm(Kind.BITVECTOR_OR, SYN_FLAG, ACK_FLAG)

# Constraint: flags should be SYN-ACK (SYN + ACK) and destination port should be 80
is_syn_ack_packet = solver.mkTerm(Kind.EQUAL, flags, syn_ack_flags)
is_dest_port_80 = solver.mkTerm(Kind.EQUAL, dest_port, solver.mkBitVector(16, 80))

# Combine constraints for a SYN-ACK packet directed to port 80
syn_ack_constraints = solver.mkTerm(Kind.AND, is_syn_ack_packet, is_dest_port_80)

print("DEBUG", syn_ack_constraints)

# Add constraints to the solver
solver.assertFormula(syn_ack_constraints)

# Check if there's a valid assignment (satisfiable model) for the constraints
if solver.checkSat().isSat():
    # Get and display values for each field
    model_source_port = solver.getValue(source_port)
    model_dest_port = solver.getValue(dest_port)
    model_sequence_number = solver.getValue(sequence_number)
    model_ack_number = solver.getValue(ack_number)
    model_data_offset_reserved_flags = solver.getValue(data_offset_reserved_flags)
    model_flags = solver.getValue(flags)
    model_window_size = solver.getValue(window_size)
    model_checksum = solver.getValue(checksum)
    model_urgent_pointer = solver.getValue(urgent_pointer)

    print("SYN-ACK Packet Model:")
    print("Source Port:", str(model_source_port))
    print("Destination Port:", str(model_dest_port))
    print("Sequence Number:", str(model_sequence_number))
    print("Acknowledgment Number:", str(model_ack_number))
    print("Data Offset + Reserved + Flags:", str(model_data_offset_reserved_flags))
    print("Flags:", str(model_flags))
    print("Window Size:", str(model_window_size))
    print("Checksum:", str(model_checksum))
    print("Urgent Pointer:", str(model_urgent_pointer))
else:
    print("No satisfiable solution found for the SYN-ACK constraints.")


    #TODO
    # How to to extract from exe code and understand how instructions works with dynamic libraries
    # Binary - deserialize/deassembler (OBJ) instructions from binary - we need to get TEXT
    # INTEL X86 deassembler with use DYNAMIC LIBRARY - exe


DEBUG (and (= flags (bvor #b00000010 #b00010000)) (= dest_port #b0000000001010000))
SYN-ACK Packet Model:
Source Port: #b0000000000000000
Destination Port: #b0000000001010000
Sequence Number: #b00000000000000000000000000000000
Acknowledgment Number: #b00000000000000000000000000000000
Data Offset + Reserved + Flags: #b00000000
Flags: #b00010010
Window Size: #b0000000000000000
Checksum: #b0000000000000000
Urgent Pointer: #b0000000000000000


In [9]:
from cvc5 import Solver, Kind

def check_tcp_packet(flags_val, dest_port_val):
    # Initialize cvc5 solver
    solver = Solver()
    solver.setOption("produce-models", "true")
    solver.setLogic("QF_BV")  # Quantifier-free bit-vector logic

    # Define TCP Header Fields as symbolic bit-vectors
    source_port = solver.mkConst(solver.mkBitVectorSort(16), "source_port")
    dest_port = solver.mkConst(solver.mkBitVectorSort(16), "dest_port")
    sequence_number = solver.mkConst(solver.mkBitVectorSort(32), "sequence_number")
    ack_number = solver.mkConst(solver.mkBitVectorSort(32), "ack_number")
    data_offset_reserved_flags = solver.mkConst(solver.mkBitVectorSort(8), "data_offset_reserved_flags")
    flags = solver.mkConst(solver.mkBitVectorSort(8), "flags")
    window_size = solver.mkConst(solver.mkBitVectorSort(16), "window_size")
    checksum = solver.mkConst(solver.mkBitVectorSort(16), "checksum")
    urgent_pointer = solver.mkConst(solver.mkBitVectorSort(16), "urgent_pointer")

    # Define flag bit values
    URG_FLAG = solver.mkBitVector(8, 0x20)  # URG flag (0b00100000)
    ACK_FLAG = solver.mkBitVector(8, 0x10)  # ACK flag (0b00010000)
    PSH_FLAG = solver.mkBitVector(8, 0x08)  # PSH flag (0b00001000)
    RST_FLAG = solver.mkBitVector(8, 0x04)  # RST flag (0b00000100)
    SYN_FLAG = solver.mkBitVector(8, 0x02)  # SYN flag (0b00000010)
    FIN_FLAG = solver.mkBitVector(8, 0x01)  # FIN flag (0b00000001)

    # Create the specified flag combination dynamically
    combined_flags = solver.mkBitVector(8, flags_val)

    # Add constraints based on function arguments
    is_flag_set = solver.mkTerm(Kind.EQUAL, flags, combined_flags)
    is_dest_port_correct = solver.mkTerm(Kind.EQUAL, dest_port, solver.mkBitVector(16, dest_port_val))

    # Combine constraints
    packet_constraints = solver.mkTerm(Kind.AND, is_flag_set, is_dest_port_correct)
    solver.assertFormula(packet_constraints)

    # Check if there's a valid assignment (satisfiable model) for the constraints
    if solver.checkSat().isSat():
        # Get and display values for each field
        model_source_port = solver.getValue(source_port)
        model_dest_port = solver.getValue(dest_port)
        model_sequence_number = solver.getValue(sequence_number)
        model_ack_number = solver.getValue(ack_number)
        model_data_offset_reserved_flags = solver.getValue(data_offset_reserved_flags)
        model_flags = solver.getValue(flags)
        model_window_size = solver.getValue(window_size)
        model_checksum = solver.getValue(checksum)
        model_urgent_pointer = solver.getValue(urgent_pointer)

        print("TCP Packet Model:")
        print("Source Port:", str(model_source_port))
        print("Destination Port:", str(model_dest_port))
        print("Sequence Number:", str(model_sequence_number))
        print("Acknowledgment Number:", str(model_ack_number))
        print("Data Offset + Reserved + Flags:", str(model_data_offset_reserved_flags))
        print("Flags:", str(model_flags))
        print("Window Size:", str(model_window_size))
        print("Checksum:", str(model_checksum))
        print("Urgent Pointer:", str(model_urgent_pointer))
    else:
        print("No satisfiable solution found for the specified TCP packet constraints.")

# Example usage
# SYN-ACK packet to port 80: SYN flag (0x02) + ACK flag (0x10) = 0x12
check_tcp_packet(flags_val=0x12, dest_port_val=80)

# FIN-ACK packet to port 443: FIN flag (0x01) + ACK flag (0x10) = 0x11
check_tcp_packet(flags_val=0x11, dest_port_val=443)


TCP Packet Model:
Source Port: #b0000000000000000
Destination Port: #b0000000001010000
Sequence Number: #b00000000000000000000000000000000
Acknowledgment Number: #b00000000000000000000000000000000
Data Offset + Reserved + Flags: #b00000000
Flags: #b00010010
Window Size: #b0000000000000000
Checksum: #b0000000000000000
Urgent Pointer: #b0000000000000000
TCP Packet Model:
Source Port: #b0000000000000000
Destination Port: #b0000000110111011
Sequence Number: #b00000000000000000000000000000000
Acknowledgment Number: #b00000000000000000000000000000000
Data Offset + Reserved + Flags: #b00000000
Flags: #b00010001
Window Size: #b0000000000000000
Checksum: #b0000000000000000
Urgent Pointer: #b0000000000000000
